In [ ]:
# default_exp __init__

# handy-dict

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# exporti
from handy_dict.apply_keyed import apply_keyed

In [ ]:
# exporti
from handy_dict.return_keyed import return_keyed

In [ ]:
# exporti
from handy_dict.core import HandyDict